In [ ]:
!pip install flask flask-ngrok


In [ ]:
pip install face_recognition opencv-python numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=32faab54675c18b1545196bf9f2148d7fbc0e6e4fb4f480db549631fd42069c0
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
!pip install pyngrok


In [ ]:
!ngrok config add-authtoken 2s4dT86XUzzCgzYI0hvFlJu3W9J_3AUWxoGSSC8JecCcfnJDc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

import os
import cv2
import face_recognition
import numpy as np
import pickle
import zipfile

app = Flask(__name__)
run_with_ngrok(app)  # This starts ngrok when the app runs

# Define paths
video_path = "/content/WhatsApp Video 2025-01-24 at 12.29.53 PM.mp4"
images_path = "/content/images"
pickle_file = "/content/face_encodings.pkl"
output_frames_path = "/content/processed_frames"

if not os.path.exists(output_frames_path):
    os.makedirs(output_frames_path)

@app.route('/')
def home():
    return '''
    <h1>Face Recognition API</h1>
    <p>Use this API to upload a video and reference images for face recognition.</p>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <label for="video">Upload Video:</label><br>
        <input type="file" name="video" required><br><br>
        <label for="images">Upload Reference Images (ZIP):</label><br>
        <input type="file" name="images" required><br><br>
        <button type="submit">Upload and Process</button>
    </form>
    '''

@app.route('/upload', methods=['POST'])
def upload_files():
    # Save uploaded files
    video = request.files['video']
    video.save(video_path)

    images_zip = request.files['images']
    images_zip_path = "/content/images.zip"
    images_zip.save(images_zip_path)

    # Unzip images
    with zipfile.ZipFile(images_zip_path, "r") as zip_ref:
        zip_ref.extractall(images_path)

    # Load or compute face encodings
    if os.path.exists(pickle_file):
        with open(pickle_file, "rb") as f:
            known_face_encodings, known_face_names = pickle.load(f)
    else:
        known_face_encodings = []
        known_face_names = []

        for filename in os.listdir(images_path):
            if filename.endswith((".jpeg", ".png", ".jpg")):
                image_path = os.path.join(images_path, filename)
                image = face_recognition.load_image_file(image_path)
                encodings = face_recognition.face_encodings(image)
                if encodings:
                    known_face_encodings.append(encodings[0])
                    known_face_names.append(os.path.splitext(filename)[0])

        with open(pickle_file, "wb") as f:
            pickle.dump((known_face_encodings, known_face_names), f)

    # Process video frames
    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_count += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if matches:
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

            top, right, bottom, left = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 99, 71), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

        output_frame_path = os.path.join(output_frames_path, f"frame_{frame_count}.jpg")
        cv2.imwrite(output_frame_path, frame)

    video_capture.release()

    # Zip processed frames
    output_zip_path = "/content/processed_frames.zip"
    with zipfile.ZipFile(output_zip_path, "w") as zipf:
        for root, _, files in os.walk(output_frames_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_frames_path))

    return jsonify({
        "message": "Processing complete!",
        "download_link": "/download"
    })

@app.route('/download')
def download():
    return '''
    <h1>Download Processed Frames</h1>
    <a href="/files/processed_frames.zip" download>Click here to download the processed frames</a>
    '''

if __name__ == '__main__':
  public_url = ngrok.connect(5000)
  print("Ngrok tunnel URL:", public_url)
  app.run()


Ngrok tunnel URL: NgrokTunnel: "https://eb35-34-16-180-7.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://eb35-34-16-180-7.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:50:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:50:02] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:51:12] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 11:51:17] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2025 12:00:07] "POST /upload HTTP/1.1" 200 -
